<a href="https://www.kaggle.com/code/berraoguz/fishclassification?scriptVersionId=203136113" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Fish Classification Using Artificial Neural Networks

Bu projede, bir balık sınıflandırma veri setini kullanarak bir yapay sinir ağı (ANN) modeli eğiteceğiz. Aşama aşama veri önişlemeden modelin eğitilmesine kadar süreci takip edeceğiz.


In [ ]:
# Gerekli kütüphanelerin yüklenmesi
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.utils import class_weight

# GPU kontrolü
print("Kullanılabilir GPU'lar:", tf.config.list_physical_devices('GPU'))


## 1. Veri Önişleme

Bu bölümde veri setimizi yükleyecek ve etiketler ile dosya yollarını içeren bir DataFrame oluşturacağız.


In [ ]:
# Label ve path listeleri
labels = []
paths = []

# Veri seti dizini
fish_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'

# os.walk ile klasörleri dolaşma
for dir_name, _, filenames in os.walk(fish_dir):
    for filename in filenames:
        if os.path.splitext(filename)[-1] == '.png':  # PNG uzantılı dosyaları al
            if 'GT' not in dir_name.split():  # 'GT' klasörlerini atla
                labels.append(os.path.split(dir_name)[-1])
                paths.append(os.path.join(dir_name, filename))

# DataFrame oluşturma
data = pd.DataFrame({
    'path': paths,
    'label': labels
})

# İlk birkaç satırı inceleme
print("Veri setinin ilk 5 satırı:")
print(data.head())


## 2. Veriyi Görselleştirme

Rastgele bir resim seçip etiketini göstereceğiz.


In [ ]:
# Rastgele bir resim seç ve göster
random_image = data.sample(1).iloc[0]
img_path = random_image['path']
img = mpimg.imread(img_path)

plt.imshow(img)
plt.title(f"Label: {random_image['label']}")
plt.show()


## 3. Veriyi Train ve Test Olarak Ayırma

Veri setimizi eğitim ve test verisi olarak ayıracağız.


In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=42)


## 4. Modelin Eğitilmesi (ANN)

Burada yapay sinir ağı modelimizi oluşturup eğiteceğiz.


In [ ]:
# ImageDataGenerator ile veri ön işleme
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_data, 
    x_col='path', 
    y_col='label', 
    target_size=(224, 224),
    batch_size=32, 
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    test_data, 
    x_col='path', 
    y_col='label', 
    target_size=(224, 224),
    batch_size=32, 
    class_mode='categorical'
)

# Class_weight hesaplama
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_data['label']),
    y=train_data['label']
)
class_weights_dict = dict(enumerate(class_weights))

# ANN modeli
model = Sequential([
    Flatten(input_shape=(224, 224, 3)),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')  
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
try:
    with tf.device('/GPU:0'):  
        steps_per_epoch = 180  
        history = model.fit(
            train_generator, 
            steps_per_epoch=steps_per_epoch,
            epochs=10,  
            validation_data=test_generator,
            class_weight=class_weights_dict  
        )
except RuntimeError as e:
    print(f"GPU kullanılamadı: {e}")


## 5. Modelin Değerlendirilmesi

Modelimizin test setindeki başarısını değerlendireceğiz.


In [ ]:
# Modelin doğrulama setindeki başarısını değerlendirme
y_pred = np.argmax(model.predict(test_generator), axis=1)
y_true = test_generator.classes

# Confusion matrix oluşturma
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
plt.title("Confusion Matrix")
plt.show()

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys(), zero_division=1))


## 6. Eğitim ve Doğrulama Kaybını Görselleştirme

Modelin eğitim ve doğrulama kaybını zamanla göreceğiz.


In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
